## Capture stdout/stderr outputs

In [ ]:
#| default_exp capture_io

In [ ]:
#| export
from io import StringIO

In [ ]:
#| export
class TeeIO:
    """
    OutStream that and also passes it to the original stream.
    """
    def __init__(self, stream):
        self._io = StringIO()
        self._stream = stream
        self._call_log = []

    def write(self, s):
        self._io.write(s)
        return self._stream.write(s)

    def getvalue(self):
        return self._io.getvalue()

    # ipykernel's OutStream class defined some other things, pass them to the parent.
    def __getattr__(self, name):
        if name in ['_stream', '_io', '_call_log']:
            # This will call object.__getattr__ on self, returning the real self.value
            return super().__getattr__(name)
        self._call_log.append(("getattr", name))
        return getattr(self._stream, name)

    def __setattr__(self, name, value):
        if name in ['_stream', '_io', '_call_log']:
            super().__setattr__(name, value)
        else:
            self._call_log.append(("setattr", name, value))
            setattr(self._stdout, name, value)

In [ ]:
import sys

In [ ]:
stdout_tee = TeeIO(sys.stdout)
stderr_tee = TeeIO(sys.stderr)

sys.stdout = stdout_tee
sys.stderr = stderr_tee

print("Hello, world!")
print("This is a test of the emergency broadcast system.", file=sys.stderr)
print("I wanted to write someting\rbut then I changed my mind.")

sys.stdout = stdout_tee._stream
sys.stderr = stderr_tee._stream

Hello, world!
but then I changed my mind.


This is a test of the emergency broadcast system.


In [ ]:
stdout_tee.getvalue(), stderr_tee.getvalue()

('I wanted to write sometingbut then I changed my mind.\n',
 'This is a test of the emergency broadcast system.\n')